In [ ]:
from tensorflow.keras import layers, Model, preprocessing
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.layers import GlobalAveragePooling2D
import numpy as np

from utils import make_predictions, evaluation, data_loader

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
train_ds, val_ds, test_ds, class_names, image_size = data_loader.load_data(RANDOM_SEED)

In [ ]:
denseNet121BaseArchitecture = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freezing the early layers (retain general features)
denseNet121BaseArchitecture.trainable = True
for layer in denseNet121BaseArchitecture.layers[:-20]:  # freeze all but last 20 layers
    layer.trainable = False

# Custom classification head
x = denseNet121BaseArchitecture.output
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(len(class_names), activation='softmax')(x)

denseNet121 = Model(inputs=denseNet121BaseArchitecture.input, outputs=predictions)

denseNet121.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

denseNet121.summary()

In [ ]:
history = denseNet121.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)
evaluation.plot_loss(history)
evaluation.plot_accuracy(history)

In [ ]:
y_true, y_pred = make_predictions.predict_test_data(denseNet121, test_ds)

In [ ]:
evaluation.make_classification_report(y_true, y_pred, class_names)

In [ ]:
evaluation.make_confusion_matrix(y_true, y_pred, class_names)